# DEFININDO OS VÉRTICES DE INICIO E FIM DE CADA GRAFO DA REDE DIVIDIDA

### IMPORTANDO O PACOTE psycopg2 QUE CONECTA O PYTHON COM O POSTGRE-SQL E O PACOTE pandas PARA ORGANIZAR AS TABELAS DE ACESSIBILIDADE

In [1]:
import psycopg2 as pg

### CONECTANDO AO BANCO DE DADOS

In [4]:
con = pg.connect(host='localhost', 
                database='dissertacao_artigo1',
                user='postgres', 
                password='admin')

cur = con.cursor() #CRIANDO UMA INSTÂNCIA PARA EXECUTAR COMANDOS EM SQL

# OBS: O servidor hospedado na máquina local será conectado no banco de dados nomeado rede_exemplo, que possui usuário postgres e senha admin.

### VENDO QUANTOS GRAFOS A NOVA MALHA POSSUI 

In [5]:
tabela_grafos = 'rede_vicosa_mp' #TABELA COM A REDE
tabela_vertices = 'rede_vicosa_mp_vertices'
sql = f'select max(id0) from {tabela_grafos}' #COMANDO EM SQL A SER EXECUTADO
cur.execute(sql) #EXECUTANDO O COMANDO CRIADO
dados_consultados = cur.fetchall() #RETORNANDO OS DADOS
id_max = dados_consultados[0][0] #ID MÁXIMO DA MALHA

### DEFININDO OS VÉRTICES DE INICIO E FIM

In [6]:
for id_i in range (1, id_max + 1):
    #CONSULTANDO O VÉRTICE INICIAL DO GRAFO i:

    sql = f'SELECT rvmp.id, rvmp.geom FROM (SELECT ST_LineInterpolatePoints(ST_LineMerge(geom), 0) as geom FROM {tabela_grafos} WHERE id0 = {id_i}) as pto, {tabela_vertices} rvmp WHERE ST_Intersects(pto.geom, rvmp.geom)' #COMANDO EM SQL A SER EXECUTADO. SERÁ SELECIONADO O VÉRTICE INICIAL DO GRAFO i

    cur.execute(sql) #EXECUTANDO O COMANDO

    dados_consultados = cur.fetchall() #RETORNANDO OS DADOS
    
    id_ini = dados_consultados[0][0]
    
    #CONSULTANDO O VÉRTICE FINAL DO GRAFO i

    sql = f'SELECT rvmp.id FROM (SELECT ST_LineInterpolatePoints(ST_LineMerge(geom), 1) as geom FROM {tabela_grafos} WHERE id0 = {id_i}) as pto, {tabela_vertices} rvmp WHERE ST_Intersects(pto.geom, rvmp.geom)' #COMANDO EM SQL A SER EXECUTADO. SERÁ SELECIONADO O VÉRTICE FINAL DO GRAFO i

    cur.execute(sql) #EXECUTANDO O COMANDO

    dados_consultados = cur.fetchall() #RETORNANDO OS DADOS
    
    id_fim = dados_consultados[0][0]

    #ATUALIZANDO A TABELA DOS GRAFOS DA REDE:

    sql = f"update {tabela_grafos} set source ='{id_ini}', target = '{id_fim}' where id0 = {id_i};" #COMANDO EM SQL A SER EXECUTADO. SERÁ ATRIBUITO A TABELA DOS GRAFOS A SOURCE E TARGET ENCONTRADA.

    cur.execute(sql) #EXECUTANDO O COMANDO

    con.commit() #FINALIZANDO A EXECUÇÃO DO COMANDO
    
    print(f'ID: {id_i} -> v_ini = {id_ini} e v_fim = {id_fim}')

cur.close() #ENCERRANDO A INSTÂNCIA CRIADA PARA A EXECUÇÃO DO COMANDO

con.close() #ENCERRANDO A CONEXÃO COM O BANCO DE DADOS

ID: 1 -> v_ini = 1 e v_fim = 1007
ID: 2 -> v_ini = 1007 e v_fim = 2
ID: 3 -> v_ini = 3 e v_fim = 1014
ID: 4 -> v_ini = 1014 e v_fim = 4
ID: 5 -> v_ini = 5 e v_fim = 1023
ID: 6 -> v_ini = 1023 e v_fim = 6
ID: 7 -> v_ini = 7 e v_fim = 1067
ID: 8 -> v_ini = 1067 e v_fim = 8
ID: 9 -> v_ini = 9 e v_fim = 1082
ID: 10 -> v_ini = 1082 e v_fim = 10
ID: 11 -> v_ini = 10 e v_fim = 1096
ID: 12 -> v_ini = 1096 e v_fim = 11
ID: 13 -> v_ini = 12 e v_fim = 1097
ID: 14 -> v_ini = 1097 e v_fim = 11
ID: 15 -> v_ini = 13 e v_fim = 1102
ID: 16 -> v_ini = 1102 e v_fim = 11
ID: 17 -> v_ini = 14 e v_fim = 1106
ID: 18 -> v_ini = 1106 e v_fim = 15
ID: 19 -> v_ini = 4 e v_fim = 1019
ID: 20 -> v_ini = 1019 e v_fim = 5
ID: 21 -> v_ini = 50 e v_fim = 1049
ID: 22 -> v_ini = 1049 e v_fim = 52
ID: 23 -> v_ini = 30 e v_fim = 1054
ID: 24 -> v_ini = 1054 e v_fim = 24
ID: 25 -> v_ini = 53 e v_fim = 1055
ID: 26 -> v_ini = 1055 e v_fim = 30
ID: 27 -> v_ini = 54 e v_fim = 1059
ID: 28 -> v_ini = 1059 e v_fim = 28
ID: 29 -> v_

### FECHANDO AS CONEXÕES

In [7]:
cur.close()
con.close()